# Toronto Neighbourhood Restaurants AND Universities, Colleges, Malls & Recreational Installations

In [159]:
import pandas as pd

In [160]:
#!conda install --yes -c conda-forge geopy

In [162]:
import requests
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [163]:
LLD_df = pd.read_csv('CS_combinedNeighbourhoodLatLongData.csv')

In [164]:
LLD_df

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,Downtown Toronto,Regent Park,43.654260,-79.360636
4,North York,Lawrence Heights,43.718518,-79.464763
...,...,...,...,...
206,Etobicoke,Kingsway Park,43.628841,-79.520999
207,Etobicoke,Mimico NW,43.628841,-79.520999
208,Etobicoke,Queensway,43.628841,-79.520999
209,Etobicoke,Royal York,43.628841,-79.520999


In [165]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
                
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [2]:
CLIENT_ID = 'your own client id' 
CLIENT_SECRET = 'your secret'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
pos=1
print(LLD_df.loc[pos,'Neighbourhood'])
neighborhood_name = LLD_df.loc[pos, 'Neighbourhood'] # neighborhood name
neighborhood_latitude = LLD_df.loc[pos, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = LLD_df.loc[pos, 'Longitude'] # neighborhood longitude value
print('Latitude and longitude values of \'{}\' are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Your credentails:
CLIENT_ID: your own client id
CLIENT_SECRET:your secret


NameError: name 'LLD_df' is not defined

In [167]:
limit = 100 # top 100 venues in given Neighbourhood within radius(500)
radius = 500 # radius is in meters
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude,neighborhood_longitude, radius, limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ZBJYO13FKGEOV24ZQTABW3C4Q2ZUE1MYTOTRABYH3FQWHHLW&client_secret=PTA1YB2SNTJD5K1LJ11FB3VWEW1RAXT1GHSD0BMAHPCLQDYV&v=20180605&ll=43.7258823,-79.31557159999998&radius=500&limit=100'

In [168]:
results = requests.get(url).json()
#results

In [169]:
venues = results['response']['groups'][0]['items']
#print(venues)

nearby_venues = json_normalize(venues) # flatten JSON into dataframe
nearby_venues.head(3)

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.address,venue.location.crossStreet,venue.location.postalCode,venue.location.city,venue.location.state
0,e-0-4c633acb86b6be9a61268e34-0,0,"[{'summary': 'This spot is popular', 'type': '...",4c633acb86b6be9a61268e34,Victoria Village Arena,43.723481,-79.315635,"[{'label': 'display', 'lat': 43.72348055545508...",267,CA,Canada,[Canada],"[{'id': '4bf58dd8d48988d185941735', 'name': 'H...",0,[],NaN,NaN,NaN,NaN,NaN
1,e-0-4bbe904a85fbb713420d7167-1,0,"[{'summary': 'This spot is popular', 'type': '...",4bbe904a85fbb713420d7167,Tim Hortons,43.725517,-79.313103,"[{'label': 'display', 'lat': 43.72551663171475...",202,CA,Canada,"[1733 Eglinton Ave East (at Bermondsey Rd), To...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],1733 Eglinton Ave East,at Bermondsey Rd,M4A 1J8,Toronto,ON
2,e-0-4f3ecce6e4b0587016b6f30d-2,0,"[{'summary': 'This spot is popular', 'type': '...",4f3ecce6e4b0587016b6f30d,Portugril,43.725819,-79.312785,"[{'label': 'display', 'lat': 43.72581876267242...",224,CA,Canada,"[1733 Eglinton Avenue East (Bermondsey), Toron...","[{'id': '4def73e84765ae376e57713a', 'name': 'P...",0,[],1733 Eglinton Avenue East,Bermondsey,NaN,Toronto,ON


In [170]:
# Filtering the columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues.head(3)

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Victoria Village Arena,"[{'id': '4bf58dd8d48988d185941735', 'name': 'H...",43.723481,-79.315635
1,Tim Hortons,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.725517,-79.313103
2,Portugril,"[{'id': '4def73e84765ae376e57713a', 'name': 'P...",43.725819,-79.312785


In [171]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.head(3)
#print(nearby_venues)

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Victoria Village Arena,Hockey Arena,43.723481,-79.315635
1,Tim Hortons,Coffee Shop,43.725517,-79.313103
2,Portugril,Portuguese Restaurant,43.725819,-79.312785


In [172]:
# clean columns
# Keeping the last string and discarding all other strings after spliting the column names
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [173]:
nearby_venues.head(60)

,name,categories,lat,lng
0,Victoria Village Arena,Hockey Arena,43.723481,-79.315635
1,Tim Hortons,Coffee Shop,43.725517,-79.313103
2,Portugril,Portuguese Restaurant,43.725819,-79.312785
3,The Frig,French Restaurant,43.727051,-79.317418
4,Eglinton Ave E & Sloane Ave/Bermondsey Rd,Intersection,43.726086,-79.313620


In [174]:
nearby_venues['categories'].value_counts()

French Restaurant        1
Coffee Shop              1
Hockey Arena             1
Intersection             1
Portuguese Restaurant    1
Name: categories, dtype: int64

### Calculating recreation places count

In [175]:
def getRecreationCount(nearby_venues):

    cat = nearby_venues['categories'].value_counts()
    str = '(Spa)|(Bank)|(Art Gallery)|(Theater)|(Park)|(Historic Site)|(Event Space)|(Cosmetics Shop)|(Performing Arts Venue)|(Antique Shop)|(Pool)|(Intersection)|(Hockey Arena)|(Hockey Arena)|(Clothing Store)|(Women\'s Store)|(Boutique)|(Shoe Store)|(Fast Food Restaurant)|(Asian Restaurant)|(French Restaurant)|(Caribbean Restaurant)|(Japanese Restaurant)|(Shopping Plaza)|(Pizza Place)|(Pharmacy)|(Convenience Store)|(Coffee Shop)|(Sports Bar)|(Supermarket)|(Sushi Restaurant)|(Fried Chicken Joint)|(Sandwich Place)|(Bridal Shop)|(Middle Eastern Restaurant)|(Steakhouse)|(Bar)|(Jazz Club)|(Seafood Restaurant)|(Seafood Restaurant)|(Noodle House)|(Dance Studio)|(Monument)|(Speakeasy)|(General Travel)|(Concert Hall)'
 
    if len(nearby_venues[nearby_venues['categories'].str.match(pat = '(Italian Restaurant)')]) == 0:
        val = nearby_venues['categories'].str.match(pat = str)
        return len(nearby_venues[val])
    
    else:
        return 0

In [180]:
# Calculating recreation places count for all neighbourhoods

listRec = []

#print(LLD_df.shape)

for pos in range(LLD_df.shape[0]):

    try:
        #pos=7
        #neighborhood_name = LLD_df.loc[pos, 'Neighbourhood'] # neighborhood name
        neighborhood_latitude = LLD_df.loc[pos, 'Latitude'] # neighborhood latitude value
        neighborhood_longitude = LLD_df.loc[pos, 'Longitude'] # neighborhood longitude value
        limit = 100 # top 100 venues in given Neighbourhood within radius(500)
        radius = 500 # radius is in meters

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude,neighborhood_longitude, radius, limit)
        results = requests.get(url).json()
        venues = results['response']['groups'][0]['items']
        nearby_venues = json_normalize(venues) # flatten JSON into dataframe
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby_venues = nearby_venues.loc[:, filtered_columns]
        nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
        nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
        #nearby_venues.head(30)
        #nearby_venues['categories'].value_counts()
        
        listRec.append(getRecreationCount(nearby_venues))
        
    except:
        listRec.append(0)
        
#listRec


(211, 2)


In [177]:
LLD_df['Recreation Places Count'] = listRec
LLD_df.drop('Latitude', axis=1, inplace=True)
LLD_df.drop('Longitude', axis=1, inplace=True)
LLD_df.drop('Borough', axis=1, inplace=True)


In [178]:
LLD_df

,Neighbourhood,Recreation Places Count
0,Parkwoods,2
1,Victoria Village,4
2,Harbourfront,0
3,Regent Park,0
4,Lawrence Heights,7
...,...,...
206,Kingsway Park,3
207,Mimico NW,3
208,Queensway,3
209,Royal York,3


In [179]:
LLD_df.to_csv('CS_RecreationData.csv', index=False)